# [데이터 전처리]

In [12]:
import pandas as pd
from sklearn.datasets import load_iris

data = load_iris()
df = pd.DataFrame(data['data'], columns=['sepal_length', 'sepal_width',
                                        'petal_length', 'petal_width'])
df['target'] = data['target']

from collections import OrderedDict
import random

from numpy import NaN, NAN, nan
import numpy as np

from datetime import datetime

## 1. 추출 및 정제

- 열 단위 추출
    - df['sepal_length']
    - df[['col1', 'col2']]


- 행 단위 추출
    - df.loc[0]
    - df.loc[[row1, row2]]
    - df.iloc[-1]
    - df.iloc[[num1, num2]]


- 슬라이싱 구문 이용
    - df[num1:num2] # 원천 데이터의 직접 변경
    - df.iloc[:, [row1, row2]] # 복사본
    - df.loc[:, [num1, num2]] # 복사본
    

- melt 메서드 이용, 넓은 데이터 이용
    - id_vars: 위치 그대로 유지할 열의 이름 지정
    - value_vars: 행으로 위치를 변경할 열의 이름 지정
    - var_name: value_vars로 위치를 변경할 열의 이름 지정
    - value_name: var_name으로 위치를 변경한 열의 데이터를 저장한 열의 이름 지정
    
    
- 자료의 형 변환
    - astype('') # str, float, category 등
    - to_numeric() # errors='' 메서드에 raise, coerce, ignore로 오류 제어 
    - 메모리 절약을 위한 downcast=''인자도 있음 # integer, signed, unsigned, float 등
    - to_frame, to_datetime 등

- groupby 메서드 결과값인 그룹 오브젝트를 저장하여 계산에 이용


- datetime 오브젝트(서로 연산이 가능)
    - datetime.now()
    - datetime.today()
    - datetime(년, 월, 일, 시, 분, 초)
    - pd.to.datetime(시간데이터, format='%d/%m/%y')
    - now = datetime.now() # now로 추출시 데이터 분할
    - nowTime = now.strftime('%Y-%m-%d %H:%M:%S')
    - .dt.year .dt.month .dt.day 활용 가능, 혹은 [0].year 같이 인덱스로 지정
    - pd.read_csv('파일경로', parse_dates=[0]) # 0열을 Datetime 형으로 바꿈
    - pd.date_range(start='2xxx-xx-xx', end='2xxx-xx-xx', freq='')

    
- replace() # 특정 값을 가진 시리즈 값 교체
- sample() # 시리즈에서 임의의 값 반환
- sort_values(by='열이름', ascending=False내림차순) # 값 정렬
- sort_index(ascending=False) # 인덱스 역순으로 정렬
- to_frame # 시리즈를 데이터프레임으로
- random.shuffle() # 데이터 섞기
- split('_')
- reset_index() # 인덱스를 새로 지정
- drop_duplicates() # 중복 데이터 제거
- value_counts()

### drop() 데이터 행과 열 삭제

In [73]:
droprow = df.drop(2)
print(droprow.shape)

dropcol = df.drop(['sepal_length'], axis=1)
print(dropcol.shape)

(149, 5)
(150, 4)


## 2. 파생 변수 생성

- 데이터 생성
    - pd.Series(['abc', 12])
    - pd.DataFrame({'colname': ['값1, 값2]}, index=['index'], columns=[colnames다시전달])


- 간단한 함수 만들기
    - def 함수이름():
    - apply(함수이름, 인자, axis=0) 결합하여 사용 # 첫 번째 인자는 자동 전달


- 날짜 데이터 생성
    - pd.to_datetime(날짜형식data, format='%Y-%m-%d')

### groupby() 이용 그룹화

In [43]:
group1 = df.groupby('target')['sepal_length', 'petal_length'].mean()
print(group1)

        sepal_length  petal_length
target                            
0              5.006         1.462
1              5.936         4.260
2              6.588         5.552


### agg() 이용 집계(여러가지 메서드를 한번에 사용 가능)

In [51]:
def my_mean(values):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
    return sum / n

agg_df1 = df.groupby('target').sepal_length.agg(my_mean)
print(agg_df1)

target
0    5.006
1    5.936
2    6.588
Name: sepal_length, dtype: float64


In [56]:
agg_df2 = df.groupby('target').sepal_length.agg([np.count_nonzero, np.mean])
print(agg_df2)

        count_nonzero   mean
target                      
0                50.0  5.006
1                50.0  5.936
2                50.0  6.588


In [58]:
agg_df3 = df.groupby('target').agg({'sepal_length': 'mean', 'petal_width': 'std'})
print(agg_df3)

        sepal_length  petal_width
target                           
0              5.006     0.105386
1              5.936     0.197753
2              6.588     0.274650


### 데이터 필터링

print(df.groupby('target').filter(lambda df: df['petal_length'].mean() >= 5))

### OrderedDict()을 이용한 순서 보장 딕셔너리

In [64]:
a1 = OrderedDict({'Name': ['Chemist', 'Statistician']})
print(a1)

OrderedDict([('Name', ['Chemist', 'Statistician'])])


drop_duplicates # 중복값이없는시리즈반환

### 시간 형식 지정자 / 시간 범위의 주기(freq)

### 원-핫-인코딩으로 나타내기(더미변수)

pandas get_dummies

## 3. 데이터 병합

- 불리언 추출
    - df['colname' > col.mean()]
    
    
- 데이터 연결
    - pd.concat([df1, df2, df3]) # 행 연결 / 주의) 시리즈 연결의 경우, 열로 연결됨
    - pd.concat([df1, df2, df3], axis = 1) # 열 연결
    - df.append(df2, ignore_index=True) # 인덱스를 0부터 다시 시작
    - df['new'] == ['n1', 'n2', 'n3', 'n4']


- 공통 행, 열만 골라 연결(내부 조인)
    - pd.concat([df1, df2, df3], join='inner'))
    - pd.concat([df1, df2, df3], axis=1, join='inner'))
    

- 외부 조인(왼쪽, 오른쪽, 완전)
    - 왼쪽 외부 조인(왼쪽 데이터프레임 모두 포함)


- df1.merge(df2, left_on='같은내용열이름', right_on='같은내용열이름')
    - left_on, right_on에 전달하는 내용은 여러개라도 상관 없음
    
- pivot_table(index=[], columns='', values='')

- np.hstack vstack dstack stack

## 4. 결측값 처리

### ※ 결측값을 무시하고 계산하려면 각 함수의 skipna를 이용

- 결측값 확인
    - abc = pd.Series([1, 2, nan, 4])
    - abc.isnull()
    - pd.notnull(abc)
 
 
- 결측값 갯수
    - df.shape[0] - df.count()
    - df.isnull().sum()
    
    
- 결측값 채우기
    - abc.fillna(0) # 특정값으로 채우기
    - abc.fillna(method='ffill') # 앞의 값으로 채우기
    - abc.fillna(method='bfill') # 뒤의 값으로 채우기
    - abc.interpolate() # 양측의 중간값으로 채우기
    
    
- 결측값 평균으로 채우기
    - abcmean = abc.mean()
    - abc.fillna(abcmean)
    
    
- 결측값 삭제
    - abc_drop = abc.dropna()
    - axis=1일 경우, Na가 있는 열 모두 삭제
    - dropna(subset=['C']) 컬럼 'C'에 결측이 있는 경우 그 행 삭제

### 결측값 최빈값으로 채우기 위한 mode 구하기

In [64]:
from collections import Counter
colors = ['red', 'blue', 'red', 'green', 'blue', 'blue']
cnt = Counter(colors)
mode = cnt.most_common(1)
mode[0][0]

'blue'

### value_counts()로 결측값 갯수 확인

In [15]:
df.petal_length.value_counts(dropna=False).head()

1.5    13
1.4    13
5.1     8
4.5     8
1.3     7
Name: petal_length, dtype: int64

## 5. 문자열

word = 'grail'

sent = 'a scratch'일 때, 각각 슬라이싱([::]) 사용 가능

- capitalize # 첫 문자를 대문자로 변환
- count # 문자열의 갯수
- startswith # 특정 문자로 시작될 경우 참
- endswith # 특정 문자로 끝날 경우 참
- find # 찾을 문자열의 첫 번째 인덱스를 반환, 실패시 -1
- index # find와 같은 역할이지만 실패시 ValueError 반환
- isalpha # 모든 문자 알파벳이면 참
- isdecimal # 모든 문자 숫자면 참
- isalnum # 모든 문자 알파벳이거나 숫자면 참
- lower # 모든 문자를 소문자로
- upper # 모든 문자를 대문자로
- replace # 다른 문자로 변환
- strip # 문자열의 맨 앞과 뒤의 빈 칸을 제거
- split # 구분자 지정 문자열 나누고, 나눈 값의 리스트 반환
- partition # split과 비슷하지만 구분자도 반환
- center # 문자열을 늘이고 가운데 정렬
- zfill # 문자열의 빈칸은 0으로 채움
- join # ' '.join(['a', 'b', 'c'])과 같이 사용
- splitlines # 행을 가진 문자열 분리

### 문자열 포매팅 1

In [29]:
var = 'flesh wound'
var2 = '!!!!'
s = "it's just a {}{}!"

print(s.format(var, var2))

"it's just a flesh wound!!!!!"

### 문자열 포매팅 1 - 숫자

In [35]:
print('Some digits of pi: {}'.format(3.14592))
print('In 2005, Lu Chao of China recited {:,} digits of pi'.format(67890)) # 쉼표
print("I remember {0:.4} or {0:.4%} of what Lu Chao recited".format(7/67890))
print("My ID number is {0:05d}".format(42))

Some digits of pi: 3.14592
In 2005, Lu Chao of China recited 67,890 digits of pi
I remember 0.0001031 or 0.0103% of what Lu Chao recited
My ID number is 00042


### 문자열 포매팅 2

In [37]:
print('Some digits of pi: %d' % 3.14592)
print('%(cont)s %(value).2f' % {'cont': 'pipi', 'value': 2.718})

Some digits of pi: 3
pipi 2.72
In 2005, Lu Chao of China recited 67,890 digits of pi
I remember 0.0001031 or 0.0103% of what Lu Chao recited
My ID number is 00042


### 문자열 포매팅 3

In [39]:
var = 'flesh wound'
print(f"It's just a {var}!")

It's just a flesh wound!


### 정규식

## 6. 데이터 저장

- 피클로 저장(읽을 때도 read_pickle 필요)
    - df.to_pickle('저장위치.pickle')
    
    
- csv나 tsv로 저장
    - df.to_csv('저장위치.csv')
    - df.to_csv('저장위치.tsv', sep='t\')
    
    
- xls 저장
    - import xlwt
    - df.to_excel('저장위치.xls')
    
    
- xlsx 저장
    - import openpyxl
    - df.to_excel('저장위치.xlsx')